# NHN AI EasyMaker SDK 사용 예제

이 노트북에서는 NHN AI EasyMaker SDK를 사용하여 배치 추론 작업을 수행하는 방법을 보여줍니다.

In [ ]:
# 필요한 라이브러리 임포트
import easymaker

In [7]:
# 필요한 변수 설정
region="kr1"
appkey="YOUR_APP_KEY"

# 인증 토큰 발급 가이드 : https://docs.nhncloud.com/ko/nhncloud/ko/public-api/api-authentication/#_1
user_access_key_id="YOUR_ACCESS_KEY_ID"
secret_access_key="YOUR_SECRET_ACCESS_KEY"
access_token=easymaker.sample.get_access_token(user_access_key_id, secret_access_key),

# 배치 추론의 결과를 업로드할 경로를 설정합니다.
# NHN Cloud Object Storage에 AI EasyMaker 시스템 계정 권한 추가 필요 : https://docs.nhncloud.com/ko/Machine%20Learning/AI%20EasyMaker/ko/console-guide/#1-nhn-cloud-object-storage-ai-easymaker
result_upload_uri = "obs://YOUR_OBS_URI"

## SDK 초기화

SDK를 사용하기 위해 초기화합니다.

In [ ]:
# SDK 초기화
easymaker.init(
    region=region,
    appkey=appkey,
    access_token=access_token,
)

In [2]:
model = easymaker.Model().create_by_model_upload_uri(
    model_type_code=easymaker.sample.classification_model.model_type,
    model_upload_uri=easymaker.sample.classification_model.model_uri,
    model_name="easymaker_model_sample",
)

[AI EasyMaker] Model create request complete. model_id: rr6o4edl5wvw
[AI EasyMaker] Model create status: CREATE_REQUESTED (0:00:00) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:00:10) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:00:20) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:00:30) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:00:40) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:00:50) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:01:00) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:01:10) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:01:20) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:01:30) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:01:40) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRES

## 배치 추론 생성

모델 ID와 입력 데이터를 사용하여 배치 추론을 생성합니다.


[파라미터]

| 이름                      | 타입    | 필수 여부 | 기본값 | 유효 범위   | 설명                                                              |
| ------------------------- | ------- | --------- | ------ | ----------- |-----------------------------------------------------------------|
| batch_inference_name      | String  | 필수      | 없음   | 최대 50자   | 배치 추론 이름                                                        |
| instance_count            | Integer | 필수      | 없음   | 1~10        | 배치 추론에 사용할 인스턴스 수                                               |
| timeout_hours             | Integer | 선택      | 720    | 1~720       | 최대 배치 추론 시간(단위: 시간)                                             |
| instance_type_name             | String  | 필수      | 없음   | 없음        | 인스턴스 타입 이름(CLI로 조회 가능)                                          |
| model_name                | String  | 필수      | 없음   | 없음        | 모델 이름(CLI로 조회 가능)                                               |
| pod_count                 | Integer | 필수      | 없음   | 1~100       | 분산 추론을 적용할 파드 수                                                 |
| batch_size                | Integer | 필수      | 없음   | 1~1000      | 동시에 처리되는 데이터 샘플의 수                                              |
| inference_timeout_seconds | Integer | 필수      | 없음   | 1~1200      | 단일 추론 요청의 최대 허용 시간                                              |
| input_data_uri            | String  | 필수      | 없음   | 최대 255자  | 입력 데이터 파일 경로(NHN Cloud Object Storage 또는 NHN Cloud NAS)         |
| input_data_type           | String  | 필수      | 없음   | JSON, JSONL | 입력 데이터의 유형                                                      |
| include_glob_pattern      | String  | 선택      | 없음   | 최대 255자  | 파일 집합을 입력 데이터에서 포함할 Glob 패턴                                     |
| exclude_glob_pattern      | String  | 선택      | 없음   | 최대 255자  | 파일 집합을 입력 데이터에서 제외할 Glob 패턴                                     |
| output_upload_uri         | String  | 필수      | 없음   | 최대 255자  | 배치 추론 결과 파일이 업로드될 경로(NHN Cloud Object Storage 또는 NHN Cloud NAS) |
| data_storage_size         | Integer | 필수      | 없음   | 300~10000   | 배치 추론에 필요한 데이터를 다운로드할 저장 공간 크기(단위: GB)                          |
| description               | String  | 선택      | 없음   | 최대 255자  | 배치 추론에 대한 설명                                                    |
| use_log                   | Boolean | 선택      | False  | True, False | Log & Crash Search 서비스에 로그를 남길지 여부                              |
| wait                      | Boolean | 선택      | True   | True, False | True: 생성이 완료된 이후 반환, False: 생성 요청 후 즉시 반환                       |

In [3]:
# 인스턴스 목록 조회
instance_type_list = easymaker.BatchInference.get_instance_type_list()
for instance in instance_type_list:
    instance.print_info()
instance_type_name = instance_type_list[0].name

{
    "id": "35a73b57-58a7-434d-aa08-5249aaa95b3e",
    "name": "m2.c2m4",
    "vcpus": 2,
    "ram": 4096,
    "disk": 0,
    "gpu": false
}
{
    "id": "edc79d63-98c3-4b77-a2d4-482d70e6b554",
    "name": "m2.c4m8",
    "vcpus": 4,
    "ram": 8192,
    "disk": 0,
    "gpu": false
}
{
    "id": "eaddf5bd-2725-409a-ac57-044935747183",
    "name": "m2.c8m16",
    "vcpus": 8,
    "ram": 16384,
    "disk": 0,
    "gpu": false
}
{
    "id": "5b45e38f-8d54-4a8b-82e8-a104e13d464e",
    "name": "m2.c16m32",
    "vcpus": 16,
    "ram": 32768,
    "disk": 0,
    "gpu": false
}
{
    "id": "f5ebd3a7-f978-477d-8dcf-2a1578e29df3",
    "name": "c2.c2m2",
    "vcpus": 2,
    "ram": 2048,
    "disk": 0,
    "gpu": false
}
{
    "id": "57865308-7e1f-4931-aaea-1c36eb053900",
    "name": "c2.c4m4",
    "vcpus": 4,
    "ram": 4096,
    "disk": 0,
    "gpu": false
}
{
    "id": "c8f6212c-c259-414c-a6bd-118eb3bb6693",
    "name": "c2.c8m8",
    "vcpus": 8,
    "ram": 8192,
    "disk": 0,
    "gpu": false
}


In [3]:
# 배치 추론 생성
batch_inference = easymaker.BatchInference().run(
    model_id=model.model_id,
    batch_inference_name="batch_inference_sample2",
    description="Easymaker batch inference sample",
    input_data_uri=easymaker.sample.classification_model.batch_inference.input_data_uri,
    input_data_type=easymaker.sample.classification_model.batch_inference.input_data_type,
    output_upload_uri=result_upload_uri,
    instance_type_name=instance_type_name,
    data_storage_size=300,
    instance_count=1,
    timeout_hours=720,
    pod_count=1,
    batch_size=32,
    inference_timeout_seconds=120,
    use_log=True,
)
batch_inference.print_info()

[AI EasyMaker] Batch Inference create request complete. batch_inference_id: cgv7qky4vr5h
[AI EasyMaker] BatchInference create status: CREATE_REQUESTED (0:00:00) Please wait...
[AI EasyMaker] BatchInference create status: CREATE_IN_PROGRESS (0:00:10) Please wait...
[AI EasyMaker] BatchInference create status: CREATE_IN_PROGRESS (0:00:20) Please wait...
[AI EasyMaker] BatchInference create status: CREATE_IN_PROGRESS (0:00:30) Please wait...
[AI EasyMaker] BatchInference create status: CREATE_IN_PROGRESS (0:00:40) Please wait...
[AI EasyMaker] BatchInference create status: CREATE_IN_PROGRESS (0:00:50) Please wait...
[AI EasyMaker] BatchInference create status: CREATE_IN_PROGRESS (0:01:00) Please wait...
[AI EasyMaker] BatchInference create status: CREATE_IN_PROGRESS (0:01:10) Please wait...
[AI EasyMaker] BatchInference create status: CREATE_IN_PROGRESS (0:01:20) Please wait...
[AI EasyMaker] BatchInference create status: CREATE_IN_PROGRESS (0:01:30) Please wait...
[AI EasyMaker] BatchInf

## 배치 추론 조회

In [4]:
# 목록 조회
batch_inference_list = easymaker.BatchInference.get_list()
for batch_inference in batch_inference_list:
    batch_inference.print_info()

{
    "description": "Easymaker batch inference sample",
    "appKey": "qV5VIb2S1hr33zTS",
    "createdDatetime": "2025-05-23T07:40:29+09:00",
    "batchInferenceId": "cgv7qky4vr5h",
    "batchInferenceName": "batch_inference_sample2",
    "batchInferenceStatusCode": "COMPLETE",
    "instanceCount": 1,
    "timeoutMinutes": 43200,
    "model": {
        "appKey": "qV5VIb2S1hr33zTS",
        "createdDatetime": "2025-05-22T18:01:57+09:00",
        "modelId": "rr6o4edl5wvw",
        "modelName": "easymaker_model_sample",
        "modelStatusCode": "ACTIVE",
        "modelTypeCode": "TENSORFLOW",
        "modelUploadUri": "obs://kr1-api-object-storage.nhncloudservice.com/v1/AUTH_ed79fa143403492fbaf3ce31f0c03314/easymaker-sample/iris-tensorflow-model"
    },
    "podCount": 1,
    "maxBatchSize": 32,
    "inferenceTimeoutSeconds": 120,
    "inputDataUri": "obs://kr1-api-object-storage.nhncloudservice.com/v1/AUTH_ed79fa143403492fbaf3ce31f0c03314/easymaker-sample/iris-inference/case_4_1000_js

In [5]:
# 단건 조회
batch_inference = easymaker.BatchInference(batch_inference_id=batch_inference.batch_inference_id)

## 배치 추론 삭제

생성된 배치 추론을 삭제합니다.

In [6]:
# 배치 추론 작업 삭제
batch_inference.delete()

[AI EasyMaker] BatchInference deletion request completed. BatchInference ID : cgv7qky4vr5h
